In [ ]:
import logging 
import os

import torch
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import yolov5
import cv2

from tqdm import tqdm

: 

In [ ]:
logger = logging.getLogger(__name__)
LOG_TEMPLATE = "\n"+"-"*10+" %s "+"-"*10

: 

### Constants
utils/constant.py

In [27]:
HOMEPATH = "/Users/jiatongyu/Desktop/njfs/" #change this to your working environment 
TEST_VID_PATH = HOMEPATH + "data/test_vid.mp4" #TODO: delete this.
"""
Constants related to data preprocessing
"""
DP_JUMP = 5
TASK_BLACK_FRAME = "black frame"    # enable to filter out black frames 
TASK_NO_PERSON = "no person"        # enable to filter out frame with no person 



### Data Preprocessing
data/dataprep.py

In [67]:
def _load_yolo5_model():
    """
    Returns:
        Yolo5 model. see https://github.com/fcakyon/yolov5-pip for documentation
    """
    model = yolov5.load('yolov5s.pt')
    model.conf = 0.25  # NMS confidence threshold
    model.iou = 0.45  # NMS IoU threshold
    model.agnostic = False  # NMS class-agnostic
    model.multi_label = False  # NMS multiple labels per box
    model.max_det = 100  # maximum number of detections per image
    return model

In [68]:
model = _load_yolo5_model()

ModuleNotFoundError: No module named 'yolov5.models.experimental'

In [61]:
def getAllFrames(
  video_path:str, # full path to video to load 
  enabled_tasks:list=[TASK_BLACK_FRAME,TASK_NO_PERSON], # list of frame types to filter out. currently support black frame & no person
  save_to_folder=False, # pickle save to local 
  folder_path:str = None, # full path to local folder to save
  ):
  """
  Args:
    video_path: full path to video to load 
    enabled_task: list of filtering tasks. Currently support black frame and no person frame 
    folder_path: full path to local folder if save_to_folder is True

  Returns:
    frames: Dict with entries 
      "good": frames that passed all the filtering tasks 
      TASK_x: frames that was filtered by 
  """

  # load video.
  logger.warning(LOG_TEMPLATE,f"loading video from path {video_path}.")
  if not os.path.exists(video_path):
    raise FileNotFoundError(f"{video_path} does not exist.")
  
  # get frames 
  count = 0
  all_frames=[]

  cap=cv2.VideoCapture(video_path)
  while(cap.isOpened()):
      count += 1
      ret, frame = cap.read()
      if ret == False: break
      if count % DP_JUMP!= 0: continue 
      all_frames.append(frame)
  cap.release()
  cv2.destroyAllWindows()

  # filter by task 
  frames = {}
  frames['good']=[]
  for task_name in enabled_tasks:
    frames[task_name]=[]
  
  if TASK_NO_PERSON in enabled_tasks:
    yolo = _load_yolo5_model()

  for frame in all_frames:
    # remove black frames 
    if TASK_BLACK_FRAME in enabled_tasks:
      if np.average(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)) < 12:
        frames[TASK_BLACK_FRAME].append(frame)
        continue 

    # remove frame without people 
    if TASK_NO_PERSON in enabled_tasks:
      predictions = yolo(frame).pred[0]
      if not 0 in predictions[:, 5]:  # person label is 0
        frames[TASK_NO_PERSON].append(frame)
        continue 
    
    # good frames 
    frames['good'].append(frame)
  
    
  # save to local 
  if save_to_folder:
    if not os.path.exists(folder_path):
      raise FileNotFoundError(f"{folder_path} does not exist.")
    logger.warning(LOG_TEMPLATE,f"Saving frames to {folder_path}.")
    raise NotImplementedError()

  return frames

### Visualization
data/visualization

In [30]:
def _visualize_one_frame(frame, boxes=None, labels=None):
    """
    visualize one frame sample and dump into "./temp_vis.png"
    Args:
        boxes: bounding boxes in list. If passing only one 
                bbox, pass it as [box]
        labels: corresponding to bbox, have to be same length. 
                
    """
    fig2 = plt.figure()
    ax2 = fig2.add_subplot(111, aspect='equal')
    ax2.imshow(frame[...,::-1])

    if boxes is not None:
        for box in boxes:
            rect = patches.Rectangle((box[0],box[1]),box[2]-box[0],box[3]-box[1],fill=False,color='turquoise')
            ax2.add_patch(rect)
            
    plt.savefig('temp_vis.png')
    return 

In [34]:
frames = getAllFrames(TEST_VID_PATH)


---------- loading video from path /Users/jiatongyu/Desktop/njfs/data/test_vid.mp4. ----------


ModuleNotFoundError: No module named 'yolov5.models.experimental'

In [78]:
all_frames = frames 
frames = all_frames['good']

### Pulling prediction box

In [79]:
model = _load_yolo5_model()

In [99]:
_visualize_one_frame(frames[200])

In [96]:
predictions = model(frames[10]).pred[0]
boxes = predictions[:, :4] # x1, y1, x2, y2
scores = predictions[:, 4]
categories = predictions[:, 5]

In [108]:
i = np.where(categories==0)
_visualize_one_frame(frames[10],boxes[i])
[box] = boxes[i]

In [115]:
mask = np.zeros(frames[0].shape, dtype=np.uint8)
print(mask.shape)
print(box)
mask = cv2.rectangle(mask,pt1=(int(box[0]),int(box[1])),pt2=(int(box[2]),int(box[3])),color=(255,255,255),thickness=-1)
res = cv2.bitwise_and(frames[10],mask)
_visualize_one_frame(res)

(720, 1080, 3)
tensor([ 35.04708,  35.33896, 338.86505, 718.84888])


In [118]:
print(len(frames))

280


In [119]:
predictions = model(frames)

In [121]:
res = predictions.pred[0]

In [126]:
len(res[:,5])

1

In [5]:
!which python

/Users/jiatongyu/Desktop/njfs/venv/bin/python


In [7]:
!which python

/Users/jiatongyu/Desktop/njfs/venv/bin/python


In [18]:
a = np.array([1,1,1,1])
np.where(a==1)

(array([0, 1, 2, 3]),)

In [3]:
from tools.detection import load_yolo5_model

ModuleNotFoundError: No module named 'tools'

In [21]:
def pull_person_bbox(frames):
    logger.warning(LOG_TEMPLATE,"pulling person bounding boxes")
    yolo = YOLO._load_yolo5_model()
    frame_slices = []
    parent_mask = np.zeros(frames[0].shape, dtype=np.uint8)
    for frame in tqdm(frames):
        pred = yolo(frame).pred[0]
        boxes = pred[:, :4] # x1, y1, x2, y2
        categories = pred[:, 5]
        if not 0 in categories:
            raise ValueError("no empty frame allowed.")
        #TODO: allow multiple people in a frame DONE 
        idxs = np.where(categories==0)
        for idx in idxs:
            box = boxes[idx]
            mask = cv2.rectangle(parent_mask,pt1=(int(box[0]),int(box[1])),pt2=(int(box[2]),int(box[3])),
                color=(255,255,255),thickness=-1)
            frame_slices.append(cv2.bitwise_and(frame,mask))
    return frame_slices
        


In [22]:
frame_slices = pull_person_bbox(frames[:20])

NameError: name 'frames' is not defined